In [ ]:
!pip install python-telegram-bot --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.3/702.3 kB 36.4 MB/s eta 0:00:00


In [ ]:
!pip install python-dotenv

In [ ]:
import logging
import json
import torch
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, ContextTypes, filters

from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
from peft import PeftModel, PeftConfig
from config2 import MODEL_NAME, MESSAGE_TEMPLATE, RESPONSE_TEMPLATE, SYSTEM_PROMPT, DATA_PATH, TELEGRAM_BOT_TOKEN


# --- Загрузка данных ---
with open(DATA_PATH, "r", encoding="utf-8") as f:
    shoes_db = json.load(f)

# --- Поиск кроссовок ---
def find_shoes(query):
    query = query.lower().strip()
    results = []

    for shoe in shoes_db:
        name = shoe["name"].lower()
        desc = shoe.get("description", "").lower()
        price = shoe["price"].replace(" ", "").replace("₽", "")

        try:
            price_num = int(price.split("—")[0].split("/")[0])
        except:
            price_num = None

        match = False

        if query in name or query in desc:
            match = True
        elif "до" in query and price_num:
            max_price = int("".join(filter(str.isdigit, query.split("до")[1].split()[0])))
            if price_num <= max_price:
                match = True

        if match:
            results.append(shoe)
            if len(results) >= 5:
                break

    return results

# --- Диалоговая система ---
class Conversation:
    def __init__(self):
        self.messages = [{"role": "system", "content": SYSTEM_PROMPT}]

    def add_user_message(self, message):
        self.messages.append({"role": "user", "content": message})

    def add_bot_message(self, message):
        self.messages.append({"role": "bot", "content": message})

    def get_prompt(self):
        prompt = ""
        for msg in self.messages:
            prompt += MESSAGE_TEMPLATE.format(**msg)
        prompt += RESPONSE_TEMPLATE
        return prompt.strip()

# --- Генерация ответа от модели ---
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=False,
    torch_dtype=torch.float16,
    device_map="auto",
    offload_folder="./offload"
)
model = PeftModel.from_pretrained(model, MODEL_NAME, torch_dtype=torch.float16)
model.eval()

def generate(prompt):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to(model.device)
    output_ids = model.generate(
        **data,
        generation_config=generation_config,
        max_new_tokens=512
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip().split("bot")[0].strip()

# --- Обработчики Telegram ---
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    context.user_data["conversation"] = Conversation()
    await update.message.reply_text("Привет! Я ваш консультант по выбору кроссовок. Напишите мне что-нибудь!")


async def echo(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_input = update.message.text
    conversation = context.user_data.get("conversation")
    if not conversation:
        conversation = Conversation()
        context.user_data["conversation"] = conversation

    # Добавляем сообщение пользователя
    conversation.add_user_message(user_input)

    # Ищем кроссовки по запросу
    found_shoes = find_shoes(user_input)

    print("Найдено совпадений:", len(found_shoes))  # <-- Добавить для отладки

    if found_shoes:
        context_str = "\nНайденные модели:\n"
        for shoe in found_shoes:
            context_str += f"""
    Название: {shoe['name']}
    Цена: {shoe['price']}
    Описание: {shoe['description']}
    Доступные размеры: {', '.join(shoe['sizes'])}
    Ссылка: {shoe['url']}
    ---
    """
        conversation.add_bot_message(context_str)
    else:
        conversation.add_bot_message("По вашему запросу ничего не найдено.")

    # Генерируем финальный ответ
    prompt = conversation.get_prompt()
    response = generate(prompt)
    conversation.add_bot_message(response)

    # Отправляем пользователю
    await update.message.reply_text(response)


# --- Запуск бота ---
async def main():
    app = ApplicationBuilder().token("my_token").build()
    app.add_handler(CommandHandler("start", start))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, echo))
    await app.run_polling()


import nest_asyncio
nest_asyncio.apply()

import asyncio
asyncio.get_event_loop().run_until_complete(main())

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Найдено совпадений: 0
Найдено совпадений: 5


ERROR:telegram.ext.Application:No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_application.py", line 1298, in process_update
    await coroutine
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_handlers/basehandler.py", line 158, in handle_update
    return await self.callback(update, context)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-2-49272114068c>", line 108, in echo
    found_shoes = find_shoes(user_input)
                  ^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-2-49272114068c>", line 36, in find_shoes
    max_price = int("".join(filter(str.isdigit, query.split("до")[1].split()[0])))
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: ''


Найдено совпадений: 0


ERROR:telegram.ext.Application:No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_application.py", line 1298, in process_update
    await coroutine
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_handlers/basehandler.py", line 158, in handle_update
    return await self.callback(update, context)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-2-49272114068c>", line 133, in echo
    await update.message.reply_text(response)
  File "/usr/local/lib/python3.11/dist-packages/telegram/_message.py", line 1829, in reply_text
    return await self.get_bot().send_message(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_extbot.py", line 2988, in send_message
    return await super().send_message(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/_bot.py", lin

Найдено совпадений: 5
Найдено совпадений: 1
